In [9]:
import numpy as np
import pandas as pd
from sklearn.model_selection import GroupShuffleSplit

# ============================================================
# EDIT THESE PATHS (like your kNN script)
# ============================================================

PSMI_CSV = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\psmi\merged_geotrace_psmi_delta_3_Noisy_threshold_5_eps01.csv"
PLM_CSV = r"C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\plm\merged_geotrace_plm_delta_3_Noisy_threshold_5_eps01.csv"

MECHS = {
    "plm": PLM_CSV,
    # "psm": PSM_CSV,
    "psmi": PSMI_CSV
}

# ============================================================
# SETTINGS
# ============================================================
TRACE_LENGTHS = [2]

GRID_G = 200          # grid resolution for HMM state space: GxG
R_NEIGH = 2           # neighbor radius for sparse transitions
EMISSION_B = 0.0005   # emission bandwidth (in degrees)

TEST_SIZE = 0.30
RANDOM_SEED = 42

# ============================================================
# COLUMN NAMES IN YOUR MERGED CSV
# ============================================================
COL_TRUE_LAT = "latitude"
COL_TRUE_LON = "longitude"
COL_ID       = "identifier"
COL_Z_LAT    = "perturbed_latitude"
COL_Z_LON    = "perturbed_longitude"

# ============================================================
# LOADING + PREP
# ============================================================
def load_merged_from_path(path: str) -> pd.DataFrame:
    df = pd.read_csv(path)

    # keep only required columns
    df = df[[COL_TRUE_LAT, COL_TRUE_LON, COL_ID, COL_Z_LAT, COL_Z_LON]].copy()
    df.rename(columns={
        COL_TRUE_LAT: "true_lat",
        COL_TRUE_LON: "true_lon",
        COL_ID: "traj_id",
        COL_Z_LAT: "z_lat",
        COL_Z_LON: "z_lon",
    }, inplace=True)

    # ensure traj_id is integer-like
    df["traj_id"] = df["traj_id"].astype(int)

    # assume rows already time-ordered within each traj_id; set an explicit time index
    df["time"] = df.groupby("traj_id").cumcount()

    return df

# ============================================================
# GRID MAPPING
# ============================================================
def fit_grid_bounds(all_true_lat, all_true_lon, G=GRID_G, pad_ratio=0.01):
    lat_min, lat_max = float(np.min(all_true_lat)), float(np.max(all_true_lat))
    lon_min, lon_max = float(np.min(all_true_lon)), float(np.max(all_true_lon))
    lat_pad = (lat_max - lat_min) * pad_ratio + 1e-12
    lon_pad = (lon_max - lon_min) * pad_ratio + 1e-12
    return (lat_min - lat_pad, lat_max + lat_pad,
            lon_min - lon_pad, lon_max + lon_pad, G)

def to_cell(lat, lon, grid_params):
    lat_min, lat_max, lon_min, lon_max, G = grid_params
    r = np.floor((lat - lat_min) / (lat_max - lat_min) * G).astype(int)
    c = np.floor((lon - lon_min) / (lon_max - lon_min) * G).astype(int)
    r = np.clip(r, 0, G - 1)
    c = np.clip(c, 0, G - 1)
    return (r * G + c).astype(int)

def build_neighbor_list(G, R):
    neigh = []
    for r in range(G):
        for c in range(G):
            r0 = max(0, r - R); r1 = min(G - 1, r + R)
            c0 = max(0, c - R); c1 = min(G - 1, c + R)
            lst = [rr * G + cc for rr in range(r0, r1 + 1)
                               for cc in range(c0, c1 + 1)]
            neigh.append(np.array(lst, dtype=int))
    return neigh

# ============================================================
# HMM COMPONENTS
# ============================================================
def estimate_transition_sparse(df_train, grid_params, R=R_NEIGH, alpha=1.0):
    """
    Build sparse transition A(prev->next) using TRUE locations, restricted to neighbor set.
    alpha is Dirichlet/Laplace smoothing within neighbor set.
    """
    _, _, _, _, G = grid_params
    neigh = build_neighbor_list(G, R)
    counts = [dict() for _ in range(G * G)]

    # IMPORTANT: enforce ordering by time within each trajectory
    df_train = df_train.sort_values(["traj_id", "time"], kind="stable")

    for _, g in df_train.groupby("traj_id", sort=False):
        s = to_cell(g["true_lat"].to_numpy(), g["true_lon"].to_numpy(), grid_params)
        for t in range(1, len(s)):
            i, j = int(s[t-1]), int(s[t])
            counts[i][j] = counts[i].get(j, 0) + 1

    A_sparse = []
    for i in range(G * G):
        js = neigh[i]
        vals = np.array([counts[i].get(int(j), 0.0) for j in js], dtype=float) + alpha
        vals /= np.sum(vals)
        A_sparse.append((js, vals))
    return A_sparse, neigh

def cell_centers(states, grid_params):
    lat_min, lat_max, lon_min, lon_max, G = grid_params
    r = states // G
    c = states % G
    return (lat_min + (r + 0.5) * (lat_max - lat_min) / G,
            lon_min + (c + 0.5) * (lon_max - lon_min) / G)

def emission_probs(zlat, zlon, cand_states, grid_params):
    clat, clon = cell_centers(cand_states, grid_params)
    d = np.sqrt((zlat - clat) ** 2 + (zlon - clon) ** 2)
    w = np.exp(-d / EMISSION_B)
    s = np.sum(w)
    return w / s if s > 0 else np.ones_like(w) / len(w)

def posterior_last_step(zlats, zlons, A_sparse, neigh, grid_params):
    """
    Forward filtering over a sliding window, returning posterior over candidate states at last time step.
    Uses a dynamic candidate expansion to keep it sparse.
    """
    # init candidates around first observation
    s0 = to_cell(np.array([zlats[0]]), np.array([zlons[0]]), grid_params)[0]
    cand = neigh[int(s0)]
    alpha_vec = emission_probs(zlats[0], zlons[0], cand, grid_params)

    for t in range(1, len(zlats)):
        # expand next candidate set
        next_set = sorted(set(int(j) for s in cand for j in neigh[int(s)]))
        next_set = np.array(next_set, dtype=int)
        idx = {s: i for i, s in enumerate(next_set)}

        pred = np.zeros(len(next_set), dtype=float)

        for i, s in enumerate(cand):
            js, ps = A_sparse[int(s)]
            # js are within neighbor list of s
            for j, p in zip(js, ps):
                pred[idx[int(j)]] += alpha_vec[i] * p

        # multiply by emission
        emis = emission_probs(zlats[t], zlons[t], next_set, grid_params)
        alpha_vec = pred * emis
        ssum = np.sum(alpha_vec)
        if ssum <= 0:
            alpha_vec = np.ones_like(alpha_vec) / len(alpha_vec)
        else:
            alpha_vec /= ssum

        cand = next_set

    return alpha_vec  # posterior over "cand" at last step

def bayes_risk_hmm(df_test, L, grid_params, A_sparse, neigh):
    """
    Risk per time t is 1 - max_s posterior(s_t | z_{t-L+1:t}).
    Average over all t and all test trajectories.
    """
    df_test = df_test.sort_values(["traj_id", "time"], kind="stable")

    risks = []
    for _, g in df_test.groupby("traj_id", sort=False):
        zlat = g["z_lat"].to_numpy()
        zlon = g["z_lon"].to_numpy()
        if len(g) < L:
            continue
        for t in range(L - 1, len(g)):
            post = posterior_last_step(
                zlat[t - L + 1 : t + 1],
                zlon[t - L + 1 : t + 1],
                A_sparse, neigh, grid_params
            )
            risks.append(1.0 - float(np.max(post)))

    return float(np.mean(risks)) if risks else np.nan


# ============================================================
# RUN
# ============================================================
data = {}
for mech, path in MECHS.items():
    df = load_merged_from_path(path)
    data[mech] = df
    print(f"{mech}: loaded {len(df):,} rows from {path}")

# shared grid bounds across all mechanisms (fair comparison)
grid_params = fit_grid_bounds(
    np.concatenate([data[m]["true_lat"].to_numpy() for m in MECHS]),
    np.concatenate([data[m]["true_lon"].to_numpy() for m in MECHS]),
)

rows = []
for mech in MECHS:
    df = data[mech]

    splitter = GroupShuffleSplit(
        n_splits=1,
        test_size=TEST_SIZE,
        random_state=RANDOM_SEED
    )
    train_idx, test_idx = next(splitter.split(df, groups=df["traj_id"]))
    df_train, df_test = df.iloc[train_idx].copy(), df.iloc[test_idx].copy()

    # estimate sparse transitions from training true states
    A_sparse, neigh = estimate_transition_sparse(df_train, grid_params, R=R_NEIGH, alpha=1.0)

    for L in TRACE_LENGTHS:
        risk = bayes_risk_hmm(df_test, L, grid_params, A_sparse, neigh)
        rows.append({"mechanism": mech, "L": L, "BayesRisk_HMM": risk})
        print(f"{mech} L={L}: BayesRisk={risk:.4f}")

res = pd.DataFrame(rows).sort_values(["mechanism", "L"])
print("\n=== RESULTS ===")
print(res.to_string(index=False))


plm: loaded 63,771 rows from C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\plm\merged_geotrace_plm_delta_3_Noisy_threshold_5_eps01.csv
psmi: loaded 63,771 rows from C:\Users\ss6365\Desktop\PrivAR_PSM_PSM_I\data\distorted_only_privacy\geotrace\delta_3_Noisy_threshold_5\perturbed_1\psmi\merged_geotrace_psmi_delta_3_Noisy_threshold_5_eps01.csv
plm L=2: BayesRisk=0.8446
psmi L=2: BayesRisk=0.8309

=== RESULTS ===
mechanism  L  BayesRisk_HMM
      plm  2       0.844585
     psmi  2       0.830917
